In [1]:
import pandas as pd
import numpy as np
import pickle as pkl

import sys
sys.path.append('../')
from utils import *

In [2]:
df = pd.read_pickle('../../data/datasets/raw-video-level-watches')
videos = pd.read_pickle('../../data/videos_raw_metadata')
title_embeddings = pd.read_pickle('../../data/embeddings/title-autoencoded')

2023-08-03 16:12:16.405220: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 16:12:16.842559: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-03 16:12:20.135938: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46692 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:01:00.0, compute capability: 8.6
2023-08-03 16:12:20.136500: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46692 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:81:00.0, compute capability: 8.6


In [3]:
def trim(vec, element, length=15):
    #  a vector to length and keep the element
    if len(vec) > length:
        vec = vec[:length]
    if element not in vec:
        vec = vec[:-1] + [element]

    return vec


In [4]:
PX = []
SX = []
Y = []
for watch in df:
    playing = watch['playing']
    selected = watch['selected']
    suggesteds = watch['upnext']

    suggesteds = trim(suggesteds, selected)
    
    if playing not in videos or selected not in videos or any([s not in videos for s in suggesteds]):
        continue

    # i = []

    # for s in suggesteds:
    #     if s == selected:
    #         i.append(1)
    #     else:
    #         i.append(0)

    
    if len(suggesteds) < 15:
        continue
    
    playing = title_embeddings[videos[playing]['snippet']['title']]
    y = title_embeddings[videos[selected]['snippet']['title']]
    suggesteds = [title_embeddings[videos[s]['snippet']['title']] for s in suggesteds]

    # suggesteds = list(np.array(suggesteds).flatten())   

    PX.append(playing)
    SX.append(suggesteds)
    Y.append(y)

In [5]:
SX[0][0].shape

TensorShape([1, 128])

In [6]:
ssx = []

for i in range(15):
    ssx.append([])


for sx in SX:
    for i in range(15):
        sx[i] = np.array(sx[i])
        sx[i] = sx[i].reshape(128)
        ssx[i].append(sx[i])
px = []

for p in PX:
    p = np.array(p)
    p = p.reshape(128)
    px.append(p)

yy = []

for y in Y:
    y = np.array(y)
    y = y.reshape(128)
    yy.append(y)

In [7]:
len(px), len(ssx[9])

(5023, 5023)

In [13]:
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Concatenate, Input
from tensorflow.keras.optimizers import SGD


playing_input = Input(shape=(128))
suggested_inputs = []

for _ in range(15):
    suggested_layer = Input(shape=(128))
    suggested_inputs.append(suggested_layer)

print('suggested inputs')

playing_layer = Dense(64, activation='relu')(playing_input)
suggested_layers = []
for sg in suggested_inputs:
    sg_layer = Dense(64, activation='relu')(sg)
    sg_drop = Dropout(0.2)(sg_layer)
    suggested_layers.append(sg_drop)

print('suggested layers')

inputs = [playing_input] + suggested_inputs

merged = Concatenate(axis=1)([playing_layer] + suggested_layers)
dense1 = Dense(512, activation='relu')(merged)
dp2 = Dropout(0.3)(dense1)
output = Dense(128, activation='tanh')(dp2)
# dense2 = keras.layers.Dense(6000, activation='sigmoid')(dense1)
# output = keras.layers.Dense(5063, activation='tanh')(dense2)
model = Model(inputs=inputs, outputs=output)

suggested inputs
suggested layers


In [14]:
model.compile(loss='cosine_similarity',
                optimizer='adam',
                metrics='cosine_similarity')

In [15]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [16]:
px = np.array(px)
ssx = [np.array(s) for s in ssx]
yy = np.array(yy)

In [74]:
from random import sample
y = yy
sx = ssx

size = len(y)
test_size = int(size * 0.2)
indexes = list(range(size))
test_indexes = sample(indexes, test_size)

pxtrain, pxtest, ytrain, ytest = [], [], [], []
sxtrain = [[] for i in sx]
sxtest = [[] for i in sx]

for i in indexes:
    if i in test_indexes:
        pxtest.append(px[i])
        ytest.append(y[i])
        for j in range(15):
            sxtest[j].append(sx[j][i])
    else:
        pxtrain.append(px[i])
        ytrain.append(y[i])
        for j in range(15):
            sxtrain[j].append(sx[j][i])

In [17]:
model.fit([px] + ssx, yy, epochs=100, batch_size=64, verbose=1, validation_split=0.2)

Epoch 1/100
63/63 [==============================] - 6s 92ms/step - loss: -0.1898 - cosine_similarity: 0.1898 - val_loss: -0.2950 - val_cosine_similarity: 0.2950
Epoch 2/100
63/63 [==============================] - 6s 91ms/step - loss: -0.3537 - cosine_similarity: 0.3537 - val_loss: -0.3743 - val_cosine_similarity: 0.3743
Epoch 3/100
63/63 [==============================] - 6s 91ms/step - loss: -0.4317 - cosine_similarity: 0.4317 - val_loss: -0.4036 - val_cosine_similarity: 0.4036
Epoch 4/100
63/63 [==============================] - 6s 91ms/step - loss: -0.4780 - cosine_similarity: 0.4780 - val_loss: -0.4146 - val_cosine_similarity: 0.4146
Epoch 5/100
63/63 [==============================] - 6s 91ms/step - loss: -0.5135 - cosine_similarity: 0.5135 - val_loss: -0.4172 - val_cosine_similarity: 0.4172
Epoch 6/100
63/63 [==============================] - 6s 91ms/step - loss: -0.5415 - cosine_similarity: 0.5415 - val_loss: -0.4212 - val_cosine_similarity: 0.4212
Epoch 7/100
63/63 [=========